In [1]:
import pandas as pd
import numpy as np

## 1. Données + Nettoyage

In [2]:
data = pd.read_excel('nmf_data.xlsx')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 449 entries, 0 to 448
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Name            449 non-null    object        
 1   Position        449 non-null    object        
 2   Date            449 non-null    datetime64[ns]
 3   Week            449 non-null    object        
 4   RSI             447 non-null    float64       
 5   Relative Power  238 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 21.2+ KB


In [3]:
data['Name'] = data['Name'].replace({n: f'Athlete_{i}' for i, n in enumerate(data['Name'].unique())})
data.isna().sum()

Name                0
Position            0
Date                0
Week                0
RSI                 2
Relative Power    211
dtype: int64

In [4]:
nmf_data = data[data['RSI'].notna()].reset_index(drop=True).copy()
nmf_data = nmf_data.sort_values(by='Date', ignore_index=True)

## 2. Coefficient de Variation cumulatif

In [5]:
def expanding_mad(serie):
    mad_values = []
    for i in range(1, len(serie) + 1):
        subset = serie[:i]
        subset_median = np.nanmedian(subset)

        if np.isnan(subset_median):
            mad = 0
        else:
            deviations = np.absolute(subset - subset_median)
            mad = np.nanmedian(deviations)
        mad_values.append(mad)
    
    return mad_values

In [6]:
nmf_data['RSI_mad'] = nmf_data.groupby('Name')['RSI'].transform(expanding_mad)
nmf_data['Relative Power_mad'] = nmf_data.groupby('Name')['Relative Power'].transform(expanding_mad)

C:\Users\raflg\AppData\Local\Temp\ipykernel_15164\2575008809.py:5: RuntimeWarning: All-NaN slice encountered
  subset_median = np.nanmedian(subset)


In [7]:
nmf_data['RSI_exp_median'] = nmf_data.groupby('Name')['RSI'].transform(lambda x: x.expanding().median())
nmf_data['Relative Power_exp_median'] = nmf_data.groupby('Name')['Relative Power'].transform(lambda x: x.dropna().expanding().median())

In [8]:
nmf_data['RSI_CV'] = round(nmf_data['RSI_mad'] / nmf_data['RSI_exp_median'] *100, 1)
nmf_data['Relative Power_CV'] = round(nmf_data['Relative Power_mad'] / nmf_data['Relative Power_exp_median'] *100, 1)

## 3. Codes couleur

In [9]:
nmf_data.loc[nmf_data['RSI'] <= (nmf_data['RSI_exp_median'] - nmf_data['RSI_mad']), 'RSI_color'] = '#ffc09f' #orange
nmf_data.loc[(nmf_data['RSI'] > (nmf_data['RSI_exp_median'] - nmf_data['RSI_mad']))
             & (nmf_data['RSI'] < (nmf_data['RSI_exp_median'] + nmf_data['RSI_mad'])), 'RSI_color'] = '#a0ced9' #bleu
nmf_data.loc[nmf_data['RSI'] >= (nmf_data['RSI_exp_median'] + nmf_data['RSI_mad']), 'RSI_color'] = '#adf7b6' #vert

nmf_data.loc[nmf_data['Relative Power'] <= (nmf_data['Relative Power_exp_median'] - nmf_data['Relative Power_mad']), 'Relative Power_color'] = '#ffc09f' #orange
nmf_data.loc[(nmf_data['Relative Power'] > (nmf_data['Relative Power_exp_median'] - nmf_data['Relative Power_mad']))
             & (nmf_data['Relative Power'] < (nmf_data['Relative Power_exp_median'] + nmf_data['Relative Power_mad'])), 'Relative Power_color'] = '#a0ced9' #bleu
nmf_data.loc[nmf_data['Relative Power'] >= (nmf_data['Relative Power_exp_median'] + nmf_data['Relative Power_mad']), 'Relative Power_color'] = '#adf7b6' #vert